In [10]:
input_file = 'input/features/hg38/numTFBSConserved/numTFBSConserved.all.bed'
df_raw =pd.read_csv(input_file,header = None,sep = '\t')#[0].str.split('\t',expand =True)
df_raw = df_raw[df_raw[1].str.isnumeric()]
#df0[[1,2]] = df0[[1,2]].apply(lambda x: x.astype(int))


AttributeError: Can only use .str accessor with string values!

In [12]:
df_raw

,0,1,2,3,4,5
0,chr1,921079,921415,TCF7,429,K562
1,chr1,984163,984499,TCF7,473,K562
2,chr1,995778,996134,TCF7,239,HepG2
3,chr1,1000694,1001508,TCF7,354,K562
4,chr1,1013171,1013708,TCF7,431,K562
...,...,...,...,...,...,...
10565625,chrX,109830711,109831041,ATM,421,HepG2
10565626,chrX,154056775,154057105,ATM,383,HepG2
10565627,chrY,11556953,11557283,ATM,448,HepG2
10565628,chrY,56685414,56685744,ATM,690,HepG2


In [11]:
df_raw[df_raw[1].str.isnumeric()]

AttributeError: Can only use .str accessor with string values!

In [4]:
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('max_colwidth', 500)

In [ ]:
import pandas as pd
df1 = pd.read_csv('input/features/hg38/DGVCount/DGVCount.all.bed.gz',sep = '\t',header = None)

df2 = pd.read_csv('notebooks/Daten/GRCh37_hg19_variants_2014-10-16_counts.bed',sep = '\t',header = None)

df1[[1,2]] = df1[[1,2]].apply(lambda x: x.astype(int))
df2[[1,2]] = df2[[1,2]].apply(lambda x: x.astype(int))


In [ ]:
df01 = df1.iloc[:,4].str.replace(',','').str.strip()
df02 = df2.iloc[:,4].str.replace(',',' ').str.strip()

df1.iloc[:,4] = pd.DataFrame([' '.join(sorted(x)) for x in df01.str.split()])
df2.iloc[:,4] = pd.DataFrame([' '.join(sorted(x)) for x in df02.str.split()])

In [ ]:
pd.concat([df1,df2]).drop_duplicates(keep=False)

In [ ]:
import pandas as pd
file = '/fast/groups/ag_kircher/ReMM/MA_Lusi/Snakemake/input/features/hg38/DGVCount/test.bed'
file = 'input/features/hg38/ISCApath/ISCApath.nstd37.GRCh38.variant_call.bed'
df = pd.read_csv(file,sep = '\t',header = None)


In [ ]:
df = pd.read_csv(file,header = None, sep = ' ')


In [ ]:
f = df[df.index.isin(fe.index.to_list())]

In [ ]:
#df[2].str.isnumeric()
df[1] = df[1].astype(dtype = str, errors = 'ignore')
df[1] = df[1].astype(dtype = int, errors = 'ignore')
df[2] = df[2].astype(dtype = str, errors = 'ignore')
df[2] = df[2].astype(dtype = int, errors = 'ignore')

In [ ]:
list(map(lambda x: x.isdigit(), df[1]))


In [ ]:
import numpy as np
ind = df.iloc[:,[1]].applymap(np.isreal).index


In [ ]:
a = pd.to_numeric(df[2], errors='coerce')
idx = a.isna()
df[~idx]

In [ ]:
import pandas as pd

output_file = "input/features/hg38/dbVARCount/dbVARCount.all.bed.gz"
input_file = "input/features/hg38/dbVARCount/dbVARCount.all.bed"

#input_file = str(snakemake.input)
#output_file = str(snakemake.output)
input_file = 'input/features/hg38/numTFBSConserved/numTFBSConserved.all.bed'
output_file = 'input/features/hg38/numTFBSConserved/numTFBSConserved.all.bed.gz'


CHROMS = ['chr'+str(s)  for s in list(range(1,23))+['Y','X']]

if output_file.split('/')[-1]=='numTFBSConserved.all.bed.gz':
    df_raw =pd.read_csv(input_file,header = None,sep = '\t')
    df_raw[1] = df_raw[1].astype(dtype = str, errors = 'ignore')
    #df_raw[1] = df_raw[1].astype(dtype = int, errors = 'ignore')
else:
    df_raw =pd.read_csv(input_file,header = None)[0].str.split('\t',expand =True)
    df_raw = df_raw[df_raw[1].str.isnumeric()]



#repl = pd.read_csv('utils/GRCh38_RefSeq2UCSC.txt', sep='\t', header= None)
#repl[0]=repl[0].str[:-3]
#repl = dict(repl.values)
#df_raw[0] = df_raw[0].str[:-3].replace(repl)


for CHR in CHROMS[:1]:
    #print(CHR)
    df0 = df_raw[df_raw[0]==CHR]
    df0 = df0[df0[1].str.isnumeric()]
    df0[[1,2]] = df0[[1,2]].apply(lambda x: x.astype(int))

    df = df0#.iloc[:1000]
    df =df.reset_index(drop = True)
    borders =pd.Series(pd.Series(df[1].to_list()+df[2].to_list()).unique()).sort_values().reset_index(drop=True)

    intervals =pd.DataFrame()

    start =[]
    end = []

    for i in range(len(borders)-1):
        start.append(borders[i])
        end.append(borders[i+1])

    intervals['start']=start
    intervals['end']=end
    intervals['variants']=''

    for i in range(len(df)):
        #if i % 20000 == 0:
            #print(i)
        s = df[1][i]
        e = df[2][i]
        intervals.loc[(intervals['start']>=s) & (intervals['end']<=e),'variants'] = intervals.loc[(intervals['start']>=s) & (intervals['end']<=e),'variants'] + df[df.columns[-1]][i]+', '
    intervals['count']=intervals['variants'].str.count(', ')

    intervals['chr'] = CHR

    intervals = intervals[['chr','start','end','count','variants']]
    intervals['variants']=intervals['variants'].str[:-2]

    all_intervals = all_intervals.append(intervals)

all_intervals['start'] = all_intervals['start'].astype(int)  
all_intervals['end'] = all_intervals['end'].astype(int)  

all_intervals = all_intervals[all_intervals['count']!=0]
all_intervals.to_csv(output_file, sep = '\t', index = None, header = None, compression="gzip")




In [17]:
df_raw[1]

0              921079
1              984163
2              995778
3             1000694
4             1013171
              ...    
10565625    109830711
10565626    154056775
10565627     11556953
10565628     56685414
10565629     56707053
Name: 1, Length: 10565630, dtype: int64

In [3]:
import pandas as pd

output_file = "input/features/hg38/DGVCount/dbVARCount.all.bed.gz"
input_file = "input/features/hg38/DGVCount/dbVARCount.all.bed.gz.vartmp"

#input_file = str(snakemake.input)
#output_file = str(snakemake.output)

CHROMS = ['chr'+str(s)  for s in list(range(1,23))+['Y','X']]

all_intervals = pd.DataFrame()
if output_file.split('/')[-1]=='numTFBSConserved.all.bed.gz':
    df_raw =pd.read_csv(input_file,header = None,sep = '\t')
    df_raw[1] = df_raw[1].astype(dtype = str, errors = 'ignore')
else:
    df_raw =pd.read_csv(input_file,header = None, compression='gzip')[0].str.split('\t',expand =True)
df_raw = df_raw[df_raw[1].str.isnumeric()]

if output_file.split('/')[-1]=='DGVCount.all.bed.gz.vartmp':
    repl = pd.read_csv('utils/GRCh38_RefSeq2UCSC.txt', sep='\t', header= None)
    repl[0]=repl[0].str[:-3]
    repl = dict(repl.values)
    df_raw[0] = df_raw[0].replace(repl)
    #df_raw[0] = df_raw[0].str[:-3].replace(repl)


for CHR in CHROMS:
    #print(CHR)
    df0 = df_raw[df_raw[0]==CHR]
    df0 = df0[df0[1].str.isnumeric()]
    df0[[1,2]] = df0[[1,2]].apply(lambda x: x.astype(int))

    df = df0#.iloc[:1000]
    df =df.reset_index(drop = True)
    borders =pd.Series(pd.Series(df[1].to_list()+df[2].to_list()).unique()).sort_values().reset_index(drop=True)

    intervals =pd.DataFrame()

    start =[]
    end = []

    for i in range(len(borders)-1):
        start.append(borders[i])
        end.append(borders[i+1])

    intervals['start']=start
    intervals['end']=end
    intervals['variants']=''

    for i in range(len(df)):
        #if i % 20000 == 0:
            #print(i)
        s = df[1][i]
        e = df[2][i]
        intervals.loc[(intervals['start']>=s) & (intervals['end']<=e),'variants'] = intervals.loc[(intervals['start']>=s) & (intervals['end']<=e),'variants'] + df[df.columns[-1]][i]+', '
    intervals['count']=intervals['variants'].str.count(', ')

    intervals['chr'] = CHR

    intervals = intervals[['chr','start','end','count','variants']]
    intervals['variants']=intervals['variants'].str[:-2]

    all_intervals = all_intervals.append(intervals)

all_intervals = all_intervals[all_intervals['count']!=0]
all_intervals.to_csv(output_file, sep = '\t', index = None, header = None, compression="gzip")




FileNotFoundError: [Errno 2] No such file or directory: 'input/features/hg38/DGVCount/dbVARCount.all.bed.gz.vartmp'